# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 10 2022 1:00PM,250506,10,0086157604,ISDIN Corporation,Released
1,Nov 10 2022 1:00PM,250506,10,0086157606,ISDIN Corporation,Released
2,Nov 10 2022 1:00PM,250506,10,0086157605,ISDIN Corporation,Released
3,Nov 10 2022 1:00PM,250506,10,0086157607,ISDIN Corporation,Released
4,Nov 10 2022 1:00PM,250506,10,0086157609,ISDIN Corporation,Released
5,Nov 10 2022 1:00PM,250506,10,0086157608,ISDIN Corporation,Released
6,Nov 10 2022 1:00PM,250506,10,0086157611,ISDIN Corporation,Released
7,Nov 10 2022 1:00PM,250506,10,0086157610,ISDIN Corporation,Released
8,Nov 10 2022 1:00PM,250506,10,0086157613,ISDIN Corporation,Released
9,Nov 10 2022 1:00PM,250506,10,0086157612,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,250502,Released,8
13,250504,Released,60
14,250506,Released,55
15,250507,Released,1
16,250508,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250502,4.0,8.0
250504,NaN,60.0
250506,NaN,55.0
250507,NaN,1.0
250508,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250363,18.0,11.0
250436,0.0,1.0
250437,0.0,6.0
250442,0.0,1.0
250475,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250363,18,11
250436,0,1
250437,0,6
250442,0,1
250475,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250363,18,11
1,250436,0,1
2,250437,0,6
3,250442,0,1
4,250475,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250363,18,11
1,250436,,1
2,250437,,6
3,250442,,1
4,250475,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 10 2022 1:00PM,250506,10,ISDIN Corporation
55,Nov 10 2022 12:55PM,250508,10,ISDIN Corporation
60,Nov 10 2022 12:51PM,250507,10,"Senseonics, Incorporated"
61,Nov 10 2022 12:36PM,250504,10,ISDIN Corporation
121,Nov 10 2022 12:04PM,250502,15,"Virtus Pharmaceuticals, LLC"
133,Nov 10 2022 11:57AM,250499,15,"Mizner Bioscience, LLC"
140,Nov 10 2022 11:18AM,250492,10,ISDIN Corporation
202,Nov 10 2022 11:04AM,250490,20,"ACI Healthcare USA, Inc."
203,Nov 10 2022 10:57AM,250363,15,"Alliance Pharma, Inc."
232,Nov 10 2022 10:00AM,250475,15,FUJIFILM Diosynth Biotechnologies Texas LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 10 2022 1:00PM,250506,10,ISDIN Corporation,,55
1,Nov 10 2022 12:55PM,250508,10,ISDIN Corporation,,5
2,Nov 10 2022 12:51PM,250507,10,"Senseonics, Incorporated",,1
3,Nov 10 2022 12:36PM,250504,10,ISDIN Corporation,,60
4,Nov 10 2022 12:04PM,250502,15,"Virtus Pharmaceuticals, LLC",4,8
5,Nov 10 2022 11:57AM,250499,15,"Mizner Bioscience, LLC",1,6
6,Nov 10 2022 11:18AM,250492,10,ISDIN Corporation,37,25
7,Nov 10 2022 11:04AM,250490,20,"ACI Healthcare USA, Inc.",1,
8,Nov 10 2022 10:57AM,250363,15,"Alliance Pharma, Inc.",18,11
9,Nov 10 2022 10:00AM,250475,15,FUJIFILM Diosynth Biotechnologies Texas LLC,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 10 2022 1:00PM,250506,10,ISDIN Corporation,55,
1,Nov 10 2022 12:55PM,250508,10,ISDIN Corporation,5,
2,Nov 10 2022 12:51PM,250507,10,"Senseonics, Incorporated",1,
3,Nov 10 2022 12:36PM,250504,10,ISDIN Corporation,60,
4,Nov 10 2022 12:04PM,250502,15,"Virtus Pharmaceuticals, LLC",8,4
5,Nov 10 2022 11:57AM,250499,15,"Mizner Bioscience, LLC",6,1
6,Nov 10 2022 11:18AM,250492,10,ISDIN Corporation,25,37
7,Nov 10 2022 11:04AM,250490,20,"ACI Healthcare USA, Inc.",,1
8,Nov 10 2022 10:57AM,250363,15,"Alliance Pharma, Inc.",11,18
9,Nov 10 2022 10:00AM,250475,15,FUJIFILM Diosynth Biotechnologies Texas LLC,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 10 2022 1:00PM,250506,10,ISDIN Corporation,55,
1,Nov 10 2022 12:55PM,250508,10,ISDIN Corporation,5,
2,Nov 10 2022 12:51PM,250507,10,"Senseonics, Incorporated",1,
3,Nov 10 2022 12:36PM,250504,10,ISDIN Corporation,60,
4,Nov 10 2022 12:04PM,250502,15,"Virtus Pharmaceuticals, LLC",8,4


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 10 2022 1:00PM,250506,10,ISDIN Corporation,55.0,NaN
1,Nov 10 2022 12:55PM,250508,10,ISDIN Corporation,5.0,NaN
2,Nov 10 2022 12:51PM,250507,10,"Senseonics, Incorporated",1.0,NaN
3,Nov 10 2022 12:36PM,250504,10,ISDIN Corporation,60.0,NaN
4,Nov 10 2022 12:04PM,250502,15,"Virtus Pharmaceuticals, LLC",8.0,4.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 10 2022 1:00PM,250506,10,ISDIN Corporation,55.0,0.0
1,Nov 10 2022 12:55PM,250508,10,ISDIN Corporation,5.0,0.0
2,Nov 10 2022 12:51PM,250507,10,"Senseonics, Incorporated",1.0,0.0
3,Nov 10 2022 12:36PM,250504,10,ISDIN Corporation,60.0,0.0
4,Nov 10 2022 12:04PM,250502,15,"Virtus Pharmaceuticals, LLC",8.0,4.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2003832,154.0,37.0
15,1001839,26.0,23.0
20,250490,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2003832,154.0,37.0
1,15,1001839,26.0,23.0
2,20,250490,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,154.0,37.0
1,15,26.0,23.0
2,20,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,154.0
1,15,Released,26.0
2,20,Released,0.0
3,10,Executing,37.0
4,15,Executing,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,20
Status,,,
Executing,37.0,23.0,1.0
Released,154.0,26.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,20
0,Executing,37.0,23.0,1.0
1,Released,154.0,26.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,20
0,Executing,37.0,23.0,1.0
1,Released,154.0,26.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()